In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import spacy

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
import pickle

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# Preprocessing and modeling

In [2]:
joined = pd.read_csv('joined data.csv')
joined.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id,product_id-2,product_color_id,attribute_name,attribute_value,file
0,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,LWAX0056,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,2019-10-06 15:31:31.730524+00,2019-12-19 20:40:30.786144+00,NaN,https://frame-store.com/products/les-second-me...,NaN,{},NaN,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual,initial_tags
1,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,LWAX0056,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,2019-10-06 15:31:31.730000+00:00,2020-04-06 23:19:53.216000+00:00,2020-04-06 23:19:53.216000+00:00,https://frame-store.com/products/les-second-me...,NaN,[],185.0,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual,initial_tags
2,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,491075,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,2019-11-11 22:22:21.664000+00:00,2020-03-25 23:24:44.823000+00:00,2020-03-23 21:06:15.953000+00:00,https://bananarepublic.gap.com/browse/product....,Everything you love about our original Madison...,[],431.0,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium,initial_tags
3,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,491075,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,2019-11-11 22:22:21.664425+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Everything you love about our original Madison...,{},NaN,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium,initial_tags
4,01E2C3YN4KQ36A0REWZJ89ZN73,FREDA SALVADOR,5229129,Ace Bootie,Edgy style and expert craftsmanship combine on...,Unknown,2020-03-01 22:37:32.169000+00:00,2020-04-15 21:46:03.512000+00:00,2020-03-18 23:00:31.558000+00:00,https://shop.nordstrom.com/s/freda-salvador-ac...,"True to size.\n2 1/4"" (57mm) heel (size 8.5)\n...",[],1051.0,01E2C3YN4KQ36A0REWZJ89ZN73,01E2C3YN56ZCJ8TN45V3EC8CPS,Primary Color,Blacks,initial_tags


In [3]:
joined.shape

(159013, 18)

In [4]:
joined[joined.attribute_name == 'pattern'].attribute_value.value_counts()

animal              436
logo                253
floral              209
plaid               198
stripevertical      129
geometric           127
stripehorizontal    102
graphic              94
dots                 64
abstract             57
stripe               43
colorblock           42
tiedye               33
tropical             24
camouflage           22
multiprint           19
pinstripe            18
paisley              17
houndstooth          12
monogram              3
Name: attribute_value, dtype: int64

In [5]:
joined[joined.attribute_name == 'print'].attribute_value.value_counts()

none    4
Name: attribute_value, dtype: int64

In [6]:
pattern = joined[joined.attribute_name == 'pattern']

In [7]:
## Remove unrelevant columns and then remove duplicates

pattern = pattern.drop(['mpn', 'created_at', 'updated_at', 'deleted_at', 'brand_canonical_url', 'labels', 
                        'bc_product_id', 'product_id-2', 'file'], axis = 1)
pattern = pattern.drop_duplicates()

In [8]:
pattern.head()

,product_id,brand,product_full_name,description,brand_category,details,product_color_id,attribute_name,attribute_value
10746,01DTJ8DQ5SQYBS5HX6TDH2BSJP,Hunting Season,Leather-Paneled Raffia Shoulder Bag,"Handwoven by local artisans, Hunting Season's ...",women:BAGS:SHOULDER_BAGS,"Drawstring top\nComposition: leather, raffia (...",01DTJ8DS7T1QXA88MF9T17ZKS9,pattern,colorblock
10764,01DTJ8DQ5SQYBS5HX6TDH2BSJP,Hunting Season,Leather-Paneled Raffia Shoulder Bag,"Handwoven by local artisans, Hunting Season's ...",women:BAGS:SHOULDER_BAGS,"Drawstring top\nComposition: leather, raffia (...",01DTJ8DQ64FXG2PTNDZX5QHX4D,pattern,colorblock
10780,01DTATGG2XJ7QTQC6A4PHQRYW5,Alexis,Calixte Pussy-Bow Chiffon Top,Alexis' 'Calixte' top has a vintage '60s feel ...,women:CLOTHING:TOPS,Tie fastenings at neck\nComposition: 100% poly...,01DTATGG356E9CF3V4TWB705BC,pattern,dots
10792,01DTATGSMSV7FJYTPVAYEHPD8Z,Derek Lam,Belted lace-trimmed printed silk-crepe maxi dress,Multicolored silk-crepe Slips on 100% silk; li...,Clothing / Dresses / Maxi,"Fits true to size, take your normal size \nTho...",01DTATGSN4RWS0SJYJJCWR8PA6,pattern,geometric
10827,01DVCRRBK29DWT82DRBVVY544X,Mother of Pearl,#NAME?,Black and white Lyocell Partially concealed sn...,Clothing / Tops / Blouses,"Fits true to size, take your normal size \nCut...",01DVCRRBKF3E3MTGQR5NYADG5N,pattern,geometric


In [9]:
pattern.shape

(1612, 9)

In [10]:
pattern.isnull().sum()

product_id             0
brand                  0
product_full_name      0
description          228
brand_category        42
details               82
product_color_id       0
attribute_name         0
attribute_value        0
dtype: int64

In [11]:
#replace null values with UNKNOWN_TOKEN
pattern['description'] = pattern['description'].fillna('UNKNOWN_TOKEN')
pattern['details'] = pattern['details'].fillna('UNKNOWN_TOKEN')
pattern['brand_category'] = pattern['brand_category'].fillna('UNKNOWN_TOKEN')

In [12]:
pattern.isnull().sum()

product_id           0
brand                0
product_full_name    0
description          0
brand_category       0
details              0
product_color_id     0
attribute_name       0
attribute_value      0
dtype: int64

In [13]:
pattern['details'] = pattern['details'].str.replace("\n", "")

### Preprocessing

In [14]:
import string 
# define a function to remove punctuation
def removePunctuation(text, punctuations=string.punctuation+"``"+"’"+"”"):
    words=nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in punctuations]
    cleanedText = " ".join(newWords)
    return cleanedText

nltk_stopwords = set(stopwords.words("English"))

# define a function to remove stopwords
def removeStopwords(text, stopwords=nltk_stopwords):
    words = nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in stopwords]
    cleanedText = " ".join(newWords)
    return cleanedText

# define a function to lemmatize all texts
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    lemmatizedWords = [lemmatizer.lemmatize(word.lower()) for word in words]
    lemmatizedText = " ".join(lemmatizedWords)
    return lemmatizedText

columns = ["brand", "product_full_name", "description", "details"]
for col in columns: 
    pattern[col] = pattern[col].apply(removePunctuation)
    pattern[col] = pattern[col].apply(removeStopwords)
    pattern[col] = pattern[col].apply(lemmatize)

# generate dummy variables for all different attribute value levels
dummies = pd.get_dummies(pattern['attribute_value'])
pattern = pd.concat([pattern, dummies], axis = 1)
pattern.reset_index(inplace = True)
pattern = pattern.drop('index', axis = 1)

### Models

In [15]:
pattern["input_doc"] = pattern.brand + " " + pattern.product_full_name + " " + pattern.description + " " + pattern.details 

In [16]:
pattern.head()

,product_id,brand,product_full_name,description,brand_category,details,product_color_id,attribute_name,attribute_value,abstract,...,multiprint,paisley,pinstripe,plaid,stripe,stripehorizontal,stripevertical,tiedye,tropical,input_doc
0,01DTJ8DQ5SQYBS5HX6TDH2BSJP,hunting season,leather-paneled raffia shoulder bag,handwoven local artisan hunting season 's shou...,women:BAGS:SHOULDER_BAGS,drawstring topcomposition leather raffia plata...,01DTJ8DS7T1QXA88MF9T17ZKS9,pattern,colorblock,0,...,0,0,0,0,0,0,0,0,0,hunting season leather-paneled raffia shoulder...
1,01DTJ8DQ5SQYBS5HX6TDH2BSJP,hunting season,leather-paneled raffia shoulder bag,handwoven local artisan hunting season 's shou...,women:BAGS:SHOULDER_BAGS,drawstring topcomposition leather raffia plata...,01DTJ8DQ64FXG2PTNDZX5QHX4D,pattern,colorblock,0,...,0,0,0,0,0,0,0,0,0,hunting season leather-paneled raffia shoulder...
2,01DTATGG2XJ7QTQC6A4PHQRYW5,alexis,calixte pussy-bow chiffon top,alexis 'calixte top vintage '60s feel somehow ...,women:CLOTHING:TOPS,tie fastening neckcomposition 100 polyesterdry...,01DTATGG356E9CF3V4TWB705BC,pattern,dots,0,...,0,0,0,0,0,0,0,0,0,alexis calixte pussy-bow chiffon top alexis 'c...
3,01DTATGSMSV7FJYTPVAYEHPD8Z,derek lam,belted lace-trimmed printed silk-crepe maxi dress,multicolored silk-crepe slip 100 silk lining 1...,Clothing / Dresses / Maxi,fit true size take normal size petite frame ma...,01DTATGSN4RWS0SJYJJCWR8PA6,pattern,geometric,0,...,0,0,0,0,0,0,0,0,0,derek lam belted lace-trimmed printed silk-cre...
4,01DVCRRBK29DWT82DRBVVY544X,mother pearl,name,black white lyocell partially concealed snap f...,Clothing / Tops / Blouses,fit true size take normal size cut relaxed fit...,01DVCRRBKF3E3MTGQR5NYADG5N,pattern,geometric,0,...,0,0,0,0,0,0,0,0,0,mother pearl name black white lyocell partiall...


### Demo modeling on 'abstract'

In [17]:
X = pattern.iloc[:, -1].values
y = pattern.loc[:, "abstract"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0, stratify = y)

In [18]:
docs = list(X_train)
labels = y_train

In [19]:
# set the vocabulary size to some number that represents the total number of unique words in your vocabulary
vocab_size = 5000

In [20]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)
#tokenizer.word_index

In [21]:
from keras.preprocessing.text import text_to_word_sequence

In [22]:
from typing import List
def integer_encode_documents(docs: List[str], tokenizer: Tokenizer)-> List[List[int]]:
    documents = []
    for d in docs:
        doc_integers = []
        for i in text_to_word_sequence(d):
            doc_integers.append(tokenizer.word_index[i])
        documents.append(doc_integers)
    return documents

In [23]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [24]:
# integer encode the documents
encoded_docs = integer_encode_documents(docs, tokenizer)

#from pprint import pprint
#pprint(encoded_docs)

In [25]:
def get_max_token_length_per_doc(docs: List[List[str]])-> int:
    return max(list(map(lambda x: len(x.split()), docs)))

In [26]:
# get the max length in terms of token length
max_length = get_max_token_length_per_doc(docs)

In [27]:
max_length

150

In [28]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print("Padded docs:", padded_docs)

Padded docs: [[  60   76  679 ...    0    0    0]
 [  56   49 1434 ...    0    0    0]
 [  56  124  303 ...    0    0    0]
 ...
 [  56  124 1683 ...    0    0    0]
 [  48 4796   37 ...    0    0    0]
 [ 216  486   19 ...    0    0    0]]


In [29]:
EMBEDDING_SIZE = 100

### Define deep learning model

In [30]:
# define the model

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
model.add(Flatten()) 

model.add(Dense(1, activation='sigmoid')) 

In [31]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          500000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 15000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 15001     
Total params: 515,001
Trainable params: 515,001
Non-trainable params: 0
_________________________________________________________________


In [32]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 99.834573


In [33]:
embedding_layer = model.layers[0]
embedding_layer.get_weights()[0].shape

(5000, 100)

In [34]:
test_docs = list(X_test)
test_labels = y_test

In [35]:
encoded_test_docs = integer_encode_documents(test_docs, tokenizer)

# pad test documents
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
print("Padded docs:", padded_test_docs)

Padded docs: [[ 177  173   28 ...    0    0    0]
 [ 911  176   43 ...    0    0    0]
 [ 177  173   95 ...    0    0    0]
 ...
 [ 656 2998  244 ...    0    0    0]
 [ 264  103   19 ...    0    0    0]
 [ 494  953  762 ...    0    0    0]]


In [36]:
loss, accuracy = model.evaluate(padded_test_docs, test_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 96.526057


In [37]:
pattern.iloc[:, 29]

0       hunting season leather-paneled raffia shoulder...
1       hunting season leather-paneled raffia shoulder...
2       alexis calixte pussy-bow chiffon top alexis 'c...
3       derek lam belted lace-trimmed printed silk-cre...
4       mother pearl name black white lyocell partiall...
                              ...                        
1607    ulla johnson natalia metallic shibori long sle...
1608    zadig voltaire paros crystal buckle ankle stra...
1609    zadig voltaire used sneaker graffiti graphic c...
1610    zadig voltaire rock star circus metallic leath...
1611    sarto franco sarto bordo slide sandal blunted ...
Name: input_doc, Length: 1612, dtype: object

### For loop to test all attribute values based on the deep learning model

In [38]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [39]:
def get_max_token_length_per_doc(docs):
    return max(list(map(lambda x: len(x.split()), docs)))

In [40]:
X = pattern.iloc[:, -1].values
Accuracy = []
pattern_values = list(pattern.columns.values[9:29])
max_length_list = []
for i in range(9, 29):
    y = pattern.iloc[:, i].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0, stratify = y)

    docs = list(X_train)
    labels = y_train

    vocab_size = 6000
    EMBEDDING_SIZE = 100

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(docs)
    tokenizer.word_index

    encoded_docs = integer_encode_documents(docs, tokenizer)

    max_length = get_max_token_length_per_doc(docs)

    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

    model = Sequential()
    model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
    model.add(Flatten()) 

    model.add(Dense(1, activation='sigmoid')) 

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    # fit the model
    model.fit(padded_docs, labels, epochs=50, verbose=0)
    
    #save model
    model.save("{}_model.h5".format(pattern_values[i-9]))

    embedding_layer = model.layers[0]
    embedding_layer.get_weights()[0].shape

    test_docs = list(X_test)
    test_labels = y_test

    encoded_test_docs = integer_encode_documents(test_docs, tokenizer)

    # pad test documents
    padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

    loss, accuracy = model.evaluate(padded_test_docs, test_labels, verbose=0) # the average accuracy is around 95%
    #save tokenizer
    with open('{}_tokenizer.pickle'.format(pattern_values[i-9]), 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    max_length_list.append(max_length)
    Accuracy.append(accuracy)

In [41]:
max_length_list

[150,
 150,
 124,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150,
 150]

In [42]:
Accuracy

[0.9652605652809143,
 0.8908188343048096,
 0.9900744557380676,
 0.985111653804779,
 0.9751861095428467,
 0.9503722190856934,
 0.940446674823761,
 0.9801489114761353,
 0.9950371980667114,
 0.9528536200523376,
 0.9975185990333557,
 0.9925558567047119,
 0.9925558567047119,
 0.9900744557380676,
 0.9553349614143372,
 0.9826302528381348,
 0.9727047085762024,
 0.9354838728904724,
 0.9826302528381348,
 0.9900744557380676]

In [43]:
d = {'pattern_value': pattern_values, 'accuracy': Accuracy}
accuracy_table = pd.DataFrame(data = d)
accuracy_table

,pattern_value,accuracy
0,abstract,0.965261
1,animal,0.890819
2,camouflage,0.990074
3,colorblock,0.985112
4,dots,0.975186
5,floral,0.950372
6,geometric,0.940447
7,graphic,0.980149
8,houndstooth,0.995037
9,logo,0.952854


# Apply our trained model on full data

In [44]:
full_data = pd.read_csv('full_data_final version.csv')

In [45]:
#replace null values with UNKNOWN_TOKEN
full_data['description'] = full_data['description'].fillna('UNKNOWN_TOKEN')
full_data['details'] = full_data['details'].fillna('UNKNOWN_TOKEN')
full_data['brand_category'] = full_data['brand_category'].fillna('UNKNOWN_TOKEN')
full_data['details'] = full_data['details'].str.replace("\n", "")

In [46]:
import string 
# define a function to remove punctuation
def removePunctuation(text, punctuations=string.punctuation+"``"+"’"+"”"):
    words=nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in punctuations]
    cleanedText = " ".join(newWords)
    return cleanedText

nltk_stopwords = set(stopwords.words("English"))

# define a function to remove stopwords
def removeStopwords(text, stopwords=nltk_stopwords):
    words = nltk.word_tokenize(text)
    newWords = [word for word in words if word.lower() not in stopwords]
    cleanedText = " ".join(newWords)
    return cleanedText

# define a function to lemmatize all texts
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    lemmatizedWords = [lemmatizer.lemmatize(word.lower()) for word in words]
    lemmatizedText = " ".join(lemmatizedWords)
    return lemmatizedText

columns = ["brand", "product_full_name", "description", "details"]
for col in columns: 
    full_data[col] = full_data[col].apply(removePunctuation)
    full_data[col] = full_data[col].apply(removeStopwords)
    full_data[col] = full_data[col].apply(lemmatize)

In [47]:
full_data['abstract'] = 0
full_data['animal'] = 0
full_data['camouflage'] = 0
full_data['colorblock'] = 0
full_data['dots'] = 0
full_data['floral'] = 0
full_data['geometric'] = 0
full_data['graphic'] = 0
full_data['houndstooth'] = 0
full_data['logo'] = 0
full_data['monogram'] = 0
full_data['multiprint'] = 0
full_data['paisley'] = 0
full_data['pinstripe'] = 0
full_data['plaid'] = 0
full_data['stripe'] = 0
full_data['stripehorizontal'] = 0
full_data['stripevertical'] = 0
full_data['tiedye'] = 0
full_data['tropical'] = 0
full_data["input_doc"] = full_data.brand+" "+full_data.product_full_name+" "+full_data.description+" "+full_data.details 

In [48]:
full_data.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,...,multiprint,paisley,pinstripe,plaid,stripe,stripehorizontal,stripevertical,tiedye,tropical,input_doc
0,01DSE9TC2DQXDG6GWKW9NMJ416,banana republic,514683,ankle-strap pump,modern pump rounded silhouette ankle strap ext...,Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0,0,0,0,0,0,0,0,0,banana republic ankle-strap pump modern pump r...
1,01DSE9SKM19XNA6SJP36JZC065,banana republic,526676,petite tie-neck top,dress jean sneaker dress tailored trouser heel...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0,0,0,0,0,0,0,0,0,banana republic petite tie-neck top dress jean...
2,01DSJX8GD4DSAP76SPR85HRCMN,loewe,4.001E+11,52mm padded leather round sunglass,padded leather cover classic round sunglass,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,...,0,0,0,0,0,0,0,0,0,loewe 52mm padded leather round sunglass padde...
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,converse,4.00012E+11,baby 's little kid 's all-star two-tone mid-to...,iconic mid-top design get added dose support p...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,...,0,0,0,0,0,0,0,0,0,converse baby 's little kid 's all-star two-to...
4,01DSK15ZD4D5A0QXA8NSD25YXE,alexander mcqueen,4.00011E+11,64mm rimless sunglass,hexagonal shade offer rimless view intricate n...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,...,0,0,0,0,0,0,0,0,0,alexander mcqueen 64mm rimless sunglass hexago...


In [49]:
full_data.shape

(48979, 34)

In [50]:
for i in range(13, 33):
    X_train = pattern.iloc[:, -1].values
    y_train = pattern.iloc[:, i-4].values
    X_test = full_data.iloc[:, -1].values

    docs = list(X_train)
    labels = y_train

    vocab_size = 6000

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(docs)
    tokenizer.word_index

    encoded_docs = integer_encode_documents(docs, tokenizer)

    max_length = get_max_token_length_per_doc(docs)

    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

    model = Sequential()
    model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=max_length))
    model.add(Flatten()) 

    model.add(Dense(1, activation='sigmoid')) 

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    # fit the model
    model.fit(padded_docs, labels, epochs=50, verbose=0)

    embedding_layer = model.layers[0]
    embedding_layer.get_weights()[0].shape

    test_docs = list(X_test)
    
    encoded_test_docs = integer_encode_documents(test_docs, tokenizer)

    # pad test documents
    padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')
    prediction = model.predict(padded_test_docs, verbose=0)
    full_data.iloc[:, i] = prediction

In [51]:
full_data.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,...,multiprint,paisley,pinstripe,plaid,stripe,stripehorizontal,stripevertical,tiedye,tropical,input_doc
0,01DSE9TC2DQXDG6GWKW9NMJ416,banana republic,514683,ankle-strap pump,modern pump rounded silhouette ankle strap ext...,Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0.000099,0.000084,0.000144,0.000062,0.000107,0.000043,0.000573,0.000028,0.000059,banana republic ankle-strap pump modern pump r...
1,01DSE9SKM19XNA6SJP36JZC065,banana republic,526676,petite tie-neck top,dress jean sneaker dress tailored trouser heel...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0.000269,0.000133,0.000145,0.000009,0.000396,0.000237,0.000079,0.000278,0.000075,banana republic petite tie-neck top dress jean...
2,01DSJX8GD4DSAP76SPR85HRCMN,loewe,4.001E+11,52mm padded leather round sunglass,padded leather cover classic round sunglass,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,...,0.000462,0.000070,0.000297,0.000971,0.000830,0.000749,0.023187,0.000339,0.000400,loewe 52mm padded leather round sunglass padde...
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,converse,4.00012E+11,baby 's little kid 's all-star two-tone mid-to...,iconic mid-top design get added dose support p...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,...,0.000276,0.000412,0.000484,0.000131,0.000739,0.000818,0.002609,0.000274,0.001216,converse baby 's little kid 's all-star two-to...
4,01DSK15ZD4D5A0QXA8NSD25YXE,alexander mcqueen,4.00011E+11,64mm rimless sunglass,hexagonal shade offer rimless view intricate n...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,...,0.000139,0.000207,0.000399,0.006177,0.001047,0.000471,0.002260,0.000474,0.000458,alexander mcqueen 64mm rimless sunglass hexago...


In [52]:
df1 = full_data.iloc[:, 13:33]
df1.head()

,abstract,animal,camouflage,colorblock,dots,floral,geometric,graphic,houndstooth,logo,monogram,multiprint,paisley,pinstripe,plaid,stripe,stripehorizontal,stripevertical,tiedye,tropical
0,0.000130,0.957906,0.000032,0.000572,0.005349,0.001032,0.000289,1.627021e-05,0.000017,0.003251,0.000346,0.000099,0.000084,0.000144,0.000062,0.000107,0.000043,0.000573,0.000028,0.000059
1,0.000087,0.013719,0.000014,0.000003,0.002515,0.031399,0.054962,4.147577e-07,0.000007,0.000040,0.000077,0.000269,0.000133,0.000145,0.000009,0.000396,0.000237,0.000079,0.000278,0.000075
2,0.004750,0.057137,0.000112,0.000772,0.010244,0.000513,0.008701,1.422589e-03,0.000227,0.152155,0.000068,0.000462,0.000070,0.000297,0.000971,0.000830,0.000749,0.023187,0.000339,0.000400
3,0.004186,0.002933,0.000041,0.000202,0.002696,0.005074,0.010570,2.462523e-04,0.000060,0.041783,0.000076,0.000276,0.000412,0.000484,0.000131,0.000739,0.000818,0.002609,0.000274,0.001216
4,0.002908,0.040849,0.000126,0.001452,0.003596,0.001033,0.009308,2.030269e-03,0.000083,0.060929,0.000064,0.000139,0.000207,0.000399,0.006177,0.001047,0.000471,0.002260,0.000474,0.000458


In [53]:
df1['patterns/prints'] = df1.idxmax(axis = 1)

In [54]:
full_data['patterns/prints'] = df1['patterns/prints']

In [55]:
full_data.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,...,paisley,pinstripe,plaid,stripe,stripehorizontal,stripevertical,tiedye,tropical,input_doc,patterns/prints
0,01DSE9TC2DQXDG6GWKW9NMJ416,banana republic,514683,ankle-strap pump,modern pump rounded silhouette ankle strap ext...,Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0.000084,0.000144,0.000062,0.000107,0.000043,0.000573,0.000028,0.000059,banana republic ankle-strap pump modern pump r...,animal
1,01DSE9SKM19XNA6SJP36JZC065,banana republic,526676,petite tie-neck top,dress jean sneaker dress tailored trouser heel...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,...,0.000133,0.000145,0.000009,0.000396,0.000237,0.000079,0.000278,0.000075,banana republic petite tie-neck top dress jean...,geometric
2,01DSJX8GD4DSAP76SPR85HRCMN,loewe,4.001E+11,52mm padded leather round sunglass,padded leather cover classic round sunglass,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,...,0.000070,0.000297,0.000971,0.000830,0.000749,0.023187,0.000339,0.000400,loewe 52mm padded leather round sunglass padde...,logo
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,converse,4.00012E+11,baby 's little kid 's all-star two-tone mid-to...,iconic mid-top design get added dose support p...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,...,0.000412,0.000484,0.000131,0.000739,0.000818,0.002609,0.000274,0.001216,converse baby 's little kid 's all-star two-to...,logo
4,01DSK15ZD4D5A0QXA8NSD25YXE,alexander mcqueen,4.00011E+11,64mm rimless sunglass,hexagonal shade offer rimless view intricate n...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,...,0.000207,0.000399,0.006177,0.001047,0.000471,0.002260,0.000474,0.000458,alexander mcqueen 64mm rimless sunglass hexago...,logo
